In [4]:
#importing libraries
import requests
import pandas as pd
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

Step 1: Collecting the data
- I will be collecting the data from the NBA API
- From there I will convert the neccessary data into a CSV and use this for our model
- We will be collecting team data from the past 5 years?

In [6]:
SEASONS=list(range(2017,2024))

In [7]:
SEASONS

[2017, 2018, 2019, 2020, 2021, 2022, 2023]

In [8]:
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(STANDINGS_DIR, exist_ok=True)
os.makedirs(SCORES_DIR, exist_ok=True)

NameError: name 'DATA_DIR' is not defined

In [ ]:
DATA_DIR='data'
STANDINGS_DIR= os.path.join(DATA_DIR,'standings')
SCORES_DIR=os.path.join(DATA_DIR,'scores')

In [ ]:
async def get_html(url, selector, sleep = 5, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep*i)

        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f'timeout error on {url}')
            continue
        else:
            break
    return html

In [ ]:
async def scrape_season(season):
    url = f'https://www.basketball-reference.com/leagues/NBA_{season}_games.html'
    html = await get_html(url,'#content .filter')

    soup = BeautifulSoup(html)
    links = soup.find_all('a')
    href = [l['href']for l in links]
    standings_pages = [f'https://basketball-reference.com{l}' for l in href]

    for url in standings_pages:
        save_path=os.path.join(STANDINGS_DIR,url.split('/')[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url,'#all_schedule')
        with open(save_path,'w+') as f:
            f.write(html)

In [ ]:
for season in SEASONS:
    await scrape_season(season)

2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
timeout error on https://www.basketball-reference.com/leagues/NBA_2018_games.html
2017-18 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2017-1

In [ ]:
standings_files = os.listdir(STANDINGS_DIR)

In [ ]:
async def scrape_game(standings_file):
    with open(standings_file,'r') as f:
        html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all('a')
    hrefs = [l.get('href') for l in links]
    box_scores = [l for l in hrefs if l and 'boxscore' in l and '.html' in l]
    box_scores = [f'https://basketball-reference.com{l}' for l in box_scores]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR,url.split('/')[-1])
        if os.path.exists(save_path):
            continue
        
        html = await get_html(url,'#content')
        if not html:
            continue
        with open(save_path,'w+') as f:
            f.write(html)

In [ ]:
standings_files=[s for s in standings_files if '.html' in s ]

In [ ]:
for f in standings_files:
    filepath = os.path.join(STANDINGS_DIR,f)

    await scrape_game(filepath)

Clippers vs Warriors, October 21, 2021 | Basketball-Reference.com
Knicks vs Magic, October 22, 2021 | Basketball-Reference.com
Pacers vs Wizards, October 22, 2021 | Basketball-Reference.com
Hornets vs Cavaliers, October 22, 2021 | Basketball-Reference.com
Raptors vs Celtics, October 22, 2021 | Basketball-Reference.com
Nets vs 76ers, October 22, 2021 | Basketball-Reference.com
timeout error on https://basketball-reference.com/boxscores/202110220HOU.html
Thunder vs Rockets, October 22, 2021 | Basketball-Reference.com
Pelicans vs Bulls, October 22, 2021 | Basketball-Reference.com
Spurs vs Nuggets, October 22, 2021 | Basketball-Reference.com
Suns vs Lakers, October 22, 2021 | Basketball-Reference.com
Jazz vs Kings, October 22, 2021 | Basketball-Reference.com
Hawks vs Cavaliers, October 23, 2021 | Basketball-Reference.com
Heat vs Pacers, October 23, 2021 | Basketball-Reference.com
Mavericks vs Raptors, October 23, 2021 | Basketball-Reference.com
Pistons vs Bulls, October 23, 2021 | Basketba